In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
from pathlib import Path
sys.path.append("python_lib")

In [21]:
import torch
import time
from tqdm.auto import tqdm
import pandas as pd
import nnsplit
from nnsplit import train, utils, models

In [4]:
paragraphs = train.xml_to_paragraphs("train_data/dewiki-20180920-corpus.xml", max_n_paragraphs=10_000)

In [5]:
pd.Series([len(x) for x in paragraphs]).mean()

746.9238

In [6]:
cpu_splitter = nnsplit.NNSplit("de", device=torch.device("cpu"))
gpu_splitter = nnsplit.NNSplit("de", device=torch.device("cuda"))

## GPU, parallel

In [11]:
start = time.time()
_ = gpu_splitter.split(paragraphs, batch_size=2048)
delta = time.time() - start

print(f"Total: {delta:.2f} s per paragraph: {(delta / 10):.2f} ms")

Total: 4.69 s per paragraph: 0.47 ms


## GPU, sequential

In [23]:
i = 0
start = time.time()

for i in tqdm(range(len(paragraphs))):
    _ = gpu_splitter.split([paragraphs[i]], batch_size=2048)
delta = time.time() - start

print(f"Total: {delta:.2f} s per paragraph: {(delta / 10):.2f} ms")


Total: 189.56 s per paragraph: 18.96 ms


## CPU, parallel

In [24]:
start = time.time()
_ = cpu_splitter.split(paragraphs, batch_size=2048)
delta = time.time() - start

print(f"Total: {delta:.2f} s per paragraph: {(delta / 10):.2f} ms")

Total: 118.90 s per paragraph: 11.89 ms


## CPU, sequential

In [ ]:
i = 0
start = time.time()

for i in tqdm(range(len(paragraphs))):
    _ = cpu_splitter.split([paragraphs[i]], batch_size=2048)
delta = time.time() - start

print(f"Total: {delta:.2f} s per paragraph: {(delta / 10):.2f} ms")